### Import packages and Gaia tables

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table, join
from astropy.coordinates import SkyCoord
from astropy import units as u

from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
#tables = Gaia.load_tables(only_names=True)
#for table in tables:
#    print(table.name)

### Load Gaia_source data

In [ ]:
meta_source = Gaia.load_table('gaiadr3.gaia_source')
meta_source
print(meta_source)

meta_galaxies = Gaia.load_table('gaiadr3.galaxy_candidates')
print(meta_galaxies)

#meta_astrophys = Gaia.load.table('gaiadr3.astrophysical_parameters_supp')
#meta_astrophys
#print(meta_astrophys)

### List gaia source data

# Search Gaia DR3 host galaxy

In [ ]:
id = "ZTF25acahpls"
ra = 272.464618
dec = -20.987513
radius_arcsec = 5


In [ ]:

query_gaia = f"""
SELECT
  DISTANCE(POINT('ICRS', {ra}, {dec}), POINT('ICRS', s.ra, s.dec)) AS ang_sep,
  s.source_id,
  s.ra, s.dec,
  s.ref_epoch,
  s.classprob_dsc_combmod_galaxy,
  gc.source_id AS galaxy_candidate_source_id
FROM gaiadr3.gaia_source AS s
LEFT JOIN gaiadr3.galaxy_candidates AS gc
  ON gc.source_id = s.source_id
WHERE 1 = CONTAINS(
  POINT('ICRS', s.ra, s.dec),
  CIRCLE('ICRS', {ra}, {dec}, {radius_arcsec}/3600.)
)
ORDER BY ang_sep ASC
"""


In [ ]:
gaia_job = Gaia.launch_job(query_gaia)
results_gaia = gaia_job.get_results()
results_gaia

In [ ]:
# Convert Astropy Table to Pandas DataFrame
gaia_df = results_gaia.to_pandas()

# Convert ang_sep from degrees to arcseconds
gaia_df['ang_sep_arcsec'] = gaia_df['ang_sep'] * 3600.0

# Reorder and rename columns
gaia_df = gaia_df[['source_id', 'ang_sep_arcsec', 'ra', 'dec', 
         'classprob_dsc_combmod_galaxy', 'galaxy_candidate_source_id']]

# Display with limited precision for readability
pd.set_option('display.precision', 3)
display(id)
display(gaia_df)


# Cross Match with 2MASS Extended Source Database
This database includes 1.6M galaxies, with some number of other extended sources.

In [ ]:
# This code searches the 2MASS point source catalog. This data is not useful for identifying host galaxies.

# Example input coordinates (RA, Dec in degrees)
input_coords = SkyCoord([ra], [dec], unit=(u.deg, u.deg), frame='icrs')

# Set Vizier to return 2MASS XSC (catalog II/246) with a search radius
Vizier.columns = ['2MASS', 'RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag', 'Qflg', 'Ndet', 'Xflg', 'Aflg', 'opt']
Vizier.ROW_LIMIT = -1  # remove row limit
radius = 5 * u.arcsec

# Query 2MASS XSC around each input coordinate
results = []
for coord in input_coords:
    result = Vizier.query_region(coord, radius=radius, catalog='II/246')
    if result:
        df = result[0].to_pandas()
        df['input_RA'] = coord.ra.deg
        df['input_Dec'] = coord.dec.deg
        results.append(df)

# Combine results into a single DataFrame
if results:
    matched_df = pd.concat(results, ignore_index=True)
    print(matched_df[['2MASS', 'RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag', 'Qflg', 'Ndet', 'Xflg', 'Aflg', 'opt']])
else:
    print("No matches found.")

In [ ]:
import pyvo  #Virtual Observatory (VO) services. Supports TAP and ADQL.

# Connect to Vizier's TAP service
tap_service = pyvo.dal.TAPService("https://tapvizier.u-strasbg.fr/TAPVizieR/tap")

# List available tables (optional sanity check)
tables = tap_service.tables
for name in tables.keys():
    if "VII/233" in name:
        print(name)

In [ ]:
# Query the 2MASS Extended Source Catalog (XSC, VII/233) with quality cuts.
# Define your coordinate variables
ra = ra
dec = dec
radius_arcsec = radius_arcsec

# Format the ADQL query using an f-string
adql_query = f"""
SELECT *
FROM "VII/233/xsc"
WHERE 1=CONTAINS(
  -- Select sources within a circular region around your target coordinate
  POINT('ICRS', "RAJ2000", "DEJ2000"),
  CIRCLE('ICRS', {ra}, {dec}, {radius_arcsec / 3600.0})
)

-- Photometric cut: include fainter galaxies down to Ks = 16
AND "K.ext" < 16

-- Morphology cut: require semi-major axis > 5 arcsec (well-resolved sources)
AND "r3sig" > 5

-- Shape cut: axis ratio > 0.5 (exclude extremely elongated or edge-on sources)
AND "Sb/a" > 0.5

-- Photometric uncertainty cut: Ks-band uncertainty < 0.1 mag
AND "e_K.ext" < 0.1

-- Reliability cut: Spa < 2 (lower values indicate better positional reliability)
AND "Spa" < 2
"""

# Run the query
job = tap_service.search(adql_query)
results = job.to_table()
results.pprint(max_width=120)

In [ ]:
# Query the 2MASS Extended Source Catalog (XSC, VII/233) without quality cuts.
# Define your coordinate variables
ra = ra
dec = dec
radius_arcsec = radius_arcsec

# Format the ADQL query using an f-string
adql_query = f"""
SELECT *
FROM "VII/233/xsc"
WHERE 1=CONTAINS(
  -- Select sources within a circular region around your target coordinate
  POINT('ICRS', "RAJ2000", "DEJ2000"),
  CIRCLE('ICRS', {ra}, {dec}, {radius_arcsec / 3600.0})
)
"""

# Run the query
job = tap_service.search(adql_query)
results = job.to_table()
results.pprint(max_width=120)